In [22]:
import streamlit as st
import faiss
import json 
import numpy as np
import pandas as pd
from utils.rag_utils import get_ollama_embedding, ollama_chat_stream

In [23]:
# Load persisted assets
embeddings = np.load("../db/anzsco_embeddings.npy")
index = faiss.read_index("../db/anzsco.index")

with open("../db/anzsco_meta.json", "r", encoding="utf-8") as f:
    id_to_meta = json.load(f)

In [37]:
# Retrieval function
def retrieve(query, k=5):
    q_emb = np.array([get_ollama_embedding(query)]).astype("float32")
    D, I = index.search(q_emb, k)
    return [id_to_meta[i] for i in I[0]]

In [33]:
q = "Managers skill level"

bb = np.array([get_ollama_embedding(q)]).astype("float32")

w = index.search(bb, 5)


In [34]:
w

(array([[184.95383, 219.85341, 225.56136, 228.9927 , 230.85936]],
       dtype=float32),
 array([[  0,  39,  13,  50, 341]]))

In [39]:
id_to_meta[1]

{'Skill Level': 'Most occupations in this sub-major group have a level of skill commensurate with a bachelor degree or higher qualification. At least five years of relevant experience may substitute for the formal qualification. In some instances relevant experience and/or on-the-job training may be required in addition to the formal qualification (ANZSCO Skill Level 1).Tasks Include:determining and setting the overall direction and objectives of organisationsdetermining and formulating policies, rules and regulations to govern the overall operations of organisationsdirecting and controlling the overall operations of organisations to ensure objectives are metmonitoring and evaluating the overall performance of organisations and adjusting policies, rules and regulations to ensure objectives are metrepresenting the organisation at official occasions, in negotiations, at conventions, seminars, public hearings and forums, and liaising between areas of responsibilitydetermining, planning an

In [49]:

user_query = "What tasks for Manager?"
retrieved = retrieve(user_query) # give wrong answer



In [50]:
context = "\n\n".join([r['doc_text'] for r in retrieved])


In [51]:
prompt = f"""You are a helpful assistant that answers questions based on the provided context.
   Use only the information from the context to answer questions. If you're unsure or the context
   doesn't contain the relevant information, say so.
   
   Context:{context}
   Question: {user_query}
   Answer:
   """
messages_with_context = [{"role": "system", "content": prompt}]

In [53]:
response = ollama_chat_stream(messages_with_context, model="llama3")

In [58]:
from ollama import Client

# 3. Setup Ollama client
ollama_client = Client()
model_name = "llama3"  # or any model you've pulled locally via Ollama

In [59]:
response = ollama_client.chat(model=model_name, messages=messages_with_context)


In [60]:
content=response['message']['content']

In [61]:
content
        

'According to the provided context, Specialist Managers are responsible for:\n\n* Developing, implementing and monitoring strategies, policies and plans for their area of control\n* Establishing and directing operational and administrative procedures\n* Directing and coordinating the allocation of resources\n* Monitoring work progress and performance, and adjusting processes and resources to keep goals on track\n* Controlling budget planning and report preparation, and monitoring and controlling expenditure for their area of control\n* Controlling selection, training and performance of staff\n* Representing the organisation in negotiations, and at conventions, seminars, public hearings and forums\n\nThese tasks are applicable to Specialist Managers, which is a minor group under the broader category of Managers.'

In [26]:
print(retrieved)

[{'Skill Level': 'Most occupations in this sub-major group have a level of skill commensurate with a bachelor degree or higher qualification. At least five years of relevant experience may substitute for the formal qualification. In some instances relevant experience and/or on-the-job training may be required in addition to the formal qualification (ANZSCO Skill Level 1).Tasks Include:developing, implementing and monitoring strategies, policies and plans for their area of controlestablishing and directing operational and administrative proceduresdirecting and coordinating the allocation of resourcesmonitoring work progress and performance, and adjusting processes and resources to keep goals on trackcontrolling budget planning and report preparation, and monitoring and controlling expenditure for their area of controlcontrolling selection, training and performance of staffrepresenting the organisation in negotiations, and at conventions, seminars, public hearings and forums', 'Tasks': '